In [1]:
import pandas as pd
import datetime
import numpy as np
from funciones import limpiar_caracteres_especiales
from funciones import categorizacion_fondos_wos, filtrar_normalizacion_funding_wos, normalizar_casa_publicacion

In [2]:
ruta_lectura = "wos/chile/"
ruta_final = "wos/chile/finales/"
lista_hojas = [1, 2]

### Trabajos WOS

In [ ]:
tmp_xls = []

for n in lista_hojas:
    xls_base = pd.read_excel(f"{ruta_lectura}chile-2010-2023({n}).xls")
    for i,row in enumerate( xls_base.to_numpy().tolist() ):
        authors_short = [ limpiar_caracteres_especiales( n.lower().strip() ) for n in row[0].split(';') ]
        authors_full = [ limpiar_caracteres_especiales( n.lower().strip() ) for n in row[1].split(';') ]
    
        source_title = limpiar_caracteres_especiales( row[2].lower().strip() )
        funding = row[15]
        funding = funding.split(';')
        funding = [ categorizacion_fondos_wos(f) for f in funding ]
        anid, inach, extranjero, corfo, otro, uni = filtrar_normalizacion_funding_wos( funding )

        year = row[30]
        doi = row[31]
        address = row[9]

        affiliation = limpiar_caracteres_especiales( row[9].lower().strip() )
        aff_paper = 'no'
        if 'chile' in affiliation:
            aff_paper = 'si'

        n_aff_chile = 0
        n_aff_inter = 0
        for person_full, person_short in zip(authors_full, authors_short):
            if any([ person_full in aff or person_short in aff for aff in affiliation.split(';') if 'chile' in aff]): 
                n_aff_chile += 1
            else:
                n_aff_inter += 1
            
        publish_house = normalizar_casa_publicacion( row[23] ) 

        tmp_xls.append([source_title, doi, year, anid, inach, extranjero, corfo, 
            uni, otro, aff_paper, n_aff_chile, n_aff_inter, publish_house])

In [9]:
fecha = datetime.datetime.now()

full_dataframe = pd.DataFrame(np.array(tmp_xls), columns=['titulo', 'doi', 'anio', 'fin-anid', 'fin-inach', 'fin-extranjero', 
        'fin-corfo', 'fin-uni', 'fin-otro', 'aff-chile', 'n-chilenos', 'n-extranjeros', 'publisher'])
full_dataframe.to_excel(f"{ruta_final}WOS todos-papers {fecha.day}.{fecha.month}.{fecha.year}.xlsx")